In [1]:
import psycopg2

class DataBase:
#     нужно разобраться как добавлять имя базы данных
    def __init__(self,dbname="vkparser", user='dbuser',password='password',host='localhost',port=5432):
        try:
            self.connect = psycopg2.connect(dbname=dbname, user=user, 
                        password=password, host=host,port=port)
            self.c = self.connect.cursor()
        except:
            print("I am unable to connect to the database") 

    
    def create_tables(self):
        query='''CREATE TABLE IF NOT EXISTS posts (
            post_id INTEGER NOT NULL,
            from_id INTEGER NOT NULL,
            owner_id INTEGER NOT NULL,
            date INTEGER NOT NULL,
            update_flag INTEGER NOT NULL,
            text TEXT NOT NULL,
            likes INTEGER NOT NULL,
            reposts INTEGER NOT NULL,
            views INTEGER NOT NULL,
            marked_as_ads INTEGER NOT NULL,
            PRIMARY KEY (post_id, owner_id)
        );'''
        self.c.execute(query)
        query='''CREATE TABLE IF NOT EXISTS comments (
            comment_id INTEGER NOT NULL,
            user_id INTEGER NOT NULL,
            post_id INTEGER NOT NULL,
            owner_id INTEGER NOT NULL,
            date INTEGER NOT NULL,
            thread_counts INTEGER NOT NULL,
            update_flag INTEGER NOT NULL,
            text TEXT NOT NULL,
            PRIMARY KEY (comment_id, user_id,post_id,owner_id),
            FOREIGN KEY (post_id, owner_id) REFERENCES posts (post_id, owner_id)
        );'''
        
        self.c.execute(query)
        self.connect.commit()
        
        
   
    def getFromTable(self, table,rows="*", query='',  limit=100000000):
        query="SELECT {} FROM {} {} LIMIT {}".format(rows,table,query,limit)
        self.c.execute(query)
        return self.c.fetchall()
    
    def getIdPosts(self, q='post_id >= 1'):
        query="SELECT post_id,owner_id FROM posts WHERE update_flag=0 AND {}".format(q)
        self.c.execute(query)
        return self.c.fetchall()
    
    def setUpdateFlag(self,post_id,owner_id):
        query="UPDATE posts SET update_flag=1 WHERE post_id={} and owner_id={} ".format(post_id,owner_id)
        self.c.execute(query)
        

   

    def add_data(self, table, data):
        for values in data:
            query="INSERT OR REPLACE INTO %s VALUES ("+self.get_format(values)+")"
            self.c.execute(query % table, values)
        self.connect.commit()
    
    def add_post(self, values):
        for data in values:
            query='''INSERT INTO posts (post_id, from_id, owner_id, date,update_flag, text, likes,reposts,views, marked_as_ads) 
            VALUES ({},{},{},{},{},'{}',{},{},{},{})
                    ON CONFLICT (post_id, owner_id) DO UPDATE SET text='{}', 
                                    likes={},
                                    reposts={},
                                    views={}
                                    
                                    
                '''.format(data['post_id'],data['from_id'],data['owner_id'],
                           data['date'],0,data['text'],
                           data['likes'],data['reposts'],data['views'],data['marked_as_ads'],
                           data['text'],data['likes'],data['reposts'],data['views']
                          )
            self.c.execute(query)
            self.connect.commit()
    
    def add_comments(self, values):
        for data in values:
            query='''INSERT INTO comments (comment_id, user_id, post_id, owner_id, date,thread_counts, update_flag, text) 
            VALUES ({},{},{},{},{},{},{},'{}')
                    ON CONFLICT (comment_id, user_id,post_id,owner_id) DO UPDATE SET text='{}', 
                                    thread_counts={}
 
                                    
                                    
                '''.format(data['comment_id'],data['from_id'],data['post_id'],data['owner_id'],
                           data['date'],data['thread_counts'],0,data['text'],data['text'],data['thread_counts']
        
                          )
            
#             print(query)
            self.c.execute(query)
            self.connect.commit()

    def get_format(self, value):
        return(', '.join(['?'] * len(value)))
    
    def test(self):
        print('work')



    def out(self):
        self.c.close()
        self.connect.close()

### Пример добавление поста

In [2]:
# db.add_post([{'post_id': 9918674,
#   'from_id': -31101527,
#   'owner_id': -31101527,
#   'date': 1601694004,
#   'text': 'В интернете набирает популярность ролик с таксистом из Новосибирска, которого автор сравнил с героем из фильма «Брат-2»...Видео заканчивается крылатой фразой из культового кино: «У вас брат в Москве есть?»...Осторожно, на видео присутствует ненормативная лексика🔞',
#   'likes': 12,
#   'reposts': 37,
#   'views': 147867,
#   'marked_as_ads': 0}])